### BentoML Example : Multiple Models with JsonInput
# Titanic Survival Prediction with Xgboost and Lightgbm

BentoML is an open-source framework for machine learning **model serving**, aiming to **bridge the gap between Data Science and DevOps.**

Data Scientists can easily package their models trained with any ML framework using BentoMl and reproduce the model for serving in production. BentoML helps with managing packaged models in the BentoML format, and allows DevOps to deploy them as online API serving endpoints or offline batch inference jobs, on any cloud platform.

Before reading this example project, be sure to check out the [Getting started guide](https://github.com/bentoml/BentoML/blob/master/guides/quick-start/bentoml-quick-start-guide.ipynb) to learn about the basic concepts in BentoML.

This is a BentoML Demo Project demonstrating how to package and serve LightBGM model for production using BentoML.

[BentoML](http://bentoml.ai) is an open source platform for machine learning model serving and deployment.

In this example, we will use scikit-learn API for both `xgboost` and `lightgbm`. In general, we can use any python model. 

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=scikit-learn&ea=scikit-learn-multiple-models&dt=scikit-learn-multiple-models)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings

warnings.filterwarnings("ignore")

In [2]:
import bentoml
import lightgbm as lgb
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Prepare Dataset
download dataset from https://www.kaggle.com/c/titanic/data

In [3]:
%%sh
mkdir data
curl https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/train.csv -o ./data/train.csv
curl https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/test.csv -o ./data/test.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 60302  100 60302    0     0   129k      0 --:--:-- --:--:-- --:--:--  128k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 28210  100 28210    0     0  86006      0 --:--:-- --:--:-- --:--:-- 86006


In [4]:
train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
y = train_df.pop("Survived")
cols = ["Pclass", "Age", "Fare", "SibSp", "Parch"]
X_train, X_test, y_train, y_test = train_test_split(
    train_df[cols], y, test_size=0.2, random_state=42
)

# Model Training

In [6]:
lgb_model = lgb.LGBMClassifier()
lgb_model.fit(X_train, y_train)

LGBMClassifier()

In [ ]:
xgb_model = xgb.sklearn.XGBRFClassifier()
xgb_model.fit(X_train, y_train)

In [8]:
models = {"xgb": xgb_model, "lgb": lgb_model}

## Create BentoService for model serving
We are going to use `JsonInput` and return the data as `JSON` object. `JSON` objects are passed as a **list**.

In [9]:
%%writefile multiple_models_titanic_bento_service.py

import json

import bentoml
import lightgbm as lgb
import pandas as pd
import xgboost as xgb
from bentoml.adapters import JsonInput
from bentoml.frameworks.sklearn import SklearnModelArtifact


@bentoml.artifacts([SklearnModelArtifact("xgb"), SklearnModelArtifact("lgb")])
@bentoml.env(
    conda_channels=["conda-forge"],
    conda_dependencies=["lightgbm==2.3.*", "pandas==1.0.*", "xgboost==1.2.*"],
)
class TitanicSurvivalPredictionService(bentoml.BentoService):
    @bentoml.api(input=JsonInput(), batch=True)
    def predict(self, datain):
        # datain is a list of a json object.
        df = pd.read_json(json.dumps(datain[0]), orient="table")

        data = df[["Pclass", "Age", "Fare", "SibSp", "Parch"]]
        result = pd.DataFrame()
        result["xgb_proba"] = self.artifacts.xgb.predict_proba(data)[:, 1]
        result["lgb_proba"] = self.artifacts.lgb.predict_proba(data)[:, 1]
        # make sure to return as a list of json
        return [result.to_json(orient="table")]

Overwriting multiple_models_titanic_bento_service.py


# Save BentoML service archive

In [10]:
# 1) import the custom BentoService defined above
from multiple_models_titanic_bento_service import TitanicSurvivalPredictionService

# 2) `pack` it with required artifacts
bento_service = TitanicSurvivalPredictionService()
bento_service.pack("xgb", xgb_model)
bento_service.pack("lgb", lgb_model)

# 3) save your BentoSerivce
saved_path = bento_service.save()

[2020-08-25 13:47:58,242] INFO - BentoService bundle 'TitanicSurvivalPredictionService:20200825134757_086746' saved to: /Users/thein/bentoml/repository/TitanicSurvivalPredictionService/20200825134757_086746


## Containerize model server with Docker


One common way of distributing this model API server for production deployment, is via Docker containers. And BentoML provides a convenient way to do that.

Note that docker is **not available in Google Colab**. You will need to download and run this notebook locally to try out this containerization with docker feature.

If you already have docker configured, simply run the follow command to product a docker container serving the IrisClassifier prediction service created above:

In [12]:
!bentoml containerize TitanicSurvivalPredictionService:latest

Sending build context to Docker daemon  551.4kB
Step 1/15 : FROM bentoml/model-server:0.8.5
 ---> 6639eed59dc6
Step 2/15 : COPY . /bento
 ---> 8916c6323930
Step 3/15 : WORKDIR /bento
 ---> Running in e8a222116d35
Removing intermediate container e8a222116d35
 ---> ba725f7ade92
Step 4/15 : ARG PIP_INDEX_URL=https://pypi.python.org/simple/
 ---> Running in e194a9e5087c
Removing intermediate container e194a9e5087c
 ---> bfe333f7f373
Step 5/15 : ARG PIP_TRUSTED_HOST=pypi.python.org
 ---> Running in dbf5f680e79d
Removing intermediate container dbf5f680e79d
 ---> 0c9fc4ae7489
Step 6/15 : ENV PIP_INDEX_URL $PIP_INDEX_URL
 ---> Running in cb44a811259f
Removing intermediate container cb44a811259f
 ---> 7a6e44b43f45
Step 7/15 : ENV PIP_TRUSTED_HOST $PIP_TRUSTED_HOST
 ---> Running in bca323581f0b
Removing intermediate container bca323581f0b
 ---> 8f1f8b11a2d2
Step 8/15 : RUN chmod +x /bento/bentoml-init.sh
 ---> Running in 3ddc42cb25c2
Removing intermediate container 3ddc42cb25c2
 ---> 9b890a10990

In [ ]:
# port forward to 7000 
!docker run -d -p 7000:5000 TitanicSurvivalPredictionService --enable-microbatch

## Load saved BentoService

bentoml.load is the API for loading a BentoML packaged model in python:

In [11]:
import json

import bentoml

bento_model = bentoml.load(saved_path)

print(bento_model.predict(X_tests.to_json(orient='table')))

[2020-08-25 13:47:58,780] WARNING - Module `multiple_models_titanic_bento_service` already loaded, using existing imported module.


# Deployment Options

If you are at a small team with limited engineering or DevOps resources, try out automated deployment with BentoML CLI, currently supporting AWS Lambda, AWS SageMaker, and Azure Functions:
- [AWS Lambda Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_lambda.html)
- [AWS SageMaker Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_sagemaker.html)
- [Azure Functions Deployment Guide](https://docs.bentoml.org/en/latest/deployment/azure_functions.html)

If the cloud platform you are working with is not on the list above, try out these step-by-step guide on manually deploying BentoML packaged model to cloud platforms:
- [AWS ECS Deployment](https://docs.bentoml.org/en/latest/deployment/aws_ecs.html)
- [Google Cloud Run Deployment](https://docs.bentoml.org/en/latest/deployment/google_cloud_run.html)
- [Azure container instance Deployment](https://docs.bentoml.org/en/latest/deployment/azure_container_instance.html)
- [Heroku Deployment](https://docs.bentoml.org/en/latest/deployment/heroku.html)

Lastly, if you have a DevOps or ML Engineering team who's operating a Kubernetes or OpenShift cluster, use the following guides as references for implementating your deployment strategy:
- [Kubernetes Deployment](https://docs.bentoml.org/en/latest/deployment/kubernetes.html)
- [Knative Deployment](https://docs.bentoml.org/en/latest/deployment/knative.html)
- [Kubeflow Deployment](https://docs.bentoml.org/en/latest/deployment/kubeflow.html)
- [KFServing Deployment](https://docs.bentoml.org/en/latest/deployment/kfserving.html)
- [Clipper.ai Deployment Guide](https://docs.bentoml.org/en/latest/deployment/clipper.html)